In [15]:
import datetime as dt
import thredds_lsasaf_utils as tlu
import geopandas as gpd
from shapely.geometry import Point

def download_data(dstart, dend, product_freq, LatLonBox):
    # Change here your user credentials
    server_user = "karpagam"
    server_passwd = "chip-chop-2025"

    # Change here the product details
    product_path = "/MSG/MLST/NETCDF/"
    product_fname = "NETCDF4_LSASAF_MSG_LST_MSG-Disk"
    NcvarsLoad = ['LST']  # list of netcdf variables to load from remote files

    # Initialize product details
    product = tlu.lsa_product(product_path, product_fname)
    product.user = server_user
    product.passwd = server_passwd

    # List of slots to be processed:
    slot_list = tlu.gen_slot_list(dstart, dend, product_freq)
    print(f"Will load:{len(slot_list)} files: {slot_list[0]} to {slot_list[-1]}")

    # Load data
    ds_full = tlu.load_product_slots_domain(product, slot_list, NcvarsLoad, LatLonBox=LatLonBox)

    # Extract the temperature DataArray
    temperature_da = ds_full['LST']

    # Stack dimensions (combine 'time', 'lat', and 'lon') and reset the index
    df = temperature_da.stack(points=('time', 'lat', 'lon')).reset_index(['time', 'lat', 'lon']).to_dataframe(name='temperature').reset_index(drop=True)

    # Add additional columns
    df['minute'] = df['time'].dt.minute  # Added minute column
    df['hour'] = df['time'].dt.hour
    df['day'] = df['time'].dt.day
    df['month'] = df['time'].dt.month
    df['year'] = df['time'].dt.year

    # Create geometry from latitude and longitude
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Set the Coordinate Reference System (CRS) - assuming WGS84 (EPSG:4326)
    gdf.set_crs(epsg=4326, inplace=True)

    return gdf


# Time period to process
dstart = dt.datetime(2024, 8, 1, 0, 0, 0)  # start slot
dend = dt.datetime(2024, 8, 2, 23, 59, 59)  # end slot

# Frequency
# product_freq = "h"  # hourly frequency

product_freq = "h"  # hourly frequency

# Define latitude/longitude domain to load [lat_min, lat_max, lon_min, lon_max]
# Kisumu, Kenya
LatLonBox = [-0.16513, 0.00695, 34.62064, 34.90344]

# Kampala, Uganda
# LatLonBox = [0.245, 0.355, 32.522, 32.635]

# Soroti, Uganda
# LatLonBox = [1.680, 1.720, 33.580, 33.640]

# Chimoio, Mozambique
# LatLonBox = [-19.050, -19.000, 33.450, 33.500]

# Pemba, Mozambique
# LatLonBox = [-13.000, -12.950, 40.500, 40.550]

# Algiers (Alger), Algeria
# LatLonBox = [36.700, 36.780, 3.050, 3.100]

# Constantine, Algeria
# LatLonBox = [36.330, 36.380, 6.600, 6.700]

# Abidjan, Côte d'Ivoire
# LatLonBox = [5.300, 5.400, -4.050, -3.900]

# Bouaké, Côte d'Ivoire
# LatLonBox = [7.650, 7.750, -5.050, -4.950]

# Loop through each day
current_day = dstart
while current_day <= dend:
    next_day = current_day + dt.timedelta(days=1) - dt.timedelta(seconds=1)

    # Download data for the current day
    gdf = download_data(current_day, next_day, product_freq, LatLonBox)

    # Create filename for GeoPackage
    gpkg_filename = f'../downloads/MSG_2ND/kisumu/KISUMU_LST_{current_day.year}-{current_day.month:02d}-{current_day.day:02d}.gpkg'

    # Save to GeoPackage
    gdf.to_file(gpkg_filename, driver="GPKG")

    print(f'GeoDataFrame saved for {current_day.date()} as {gpkg_filename}')

    # Move to the next day
    current_day = next_day + dt.timedelta(seconds=1)

Will load:24 files: 2024-08-01 00:00:00 to 2024-08-01 23:00:00


Processing 2024-08-01 23:00:00: 100%|██████████| 24/24 [00:18<00:00,  1.28it/s]


Loaded 24 out of 24 slots with dims: FrozenMappingWarningOnValuesAccess({'time': 24, 'lat': 4, 'lon': 6}) in 18.69 seconds 
GeoDataFrame saved for 2024-08-01 as ../downloads/MSG_2ND/kisumu/KISUMU_LST_2024-08-01.gpkg
Will load:24 files: 2024-08-02 00:00:00 to 2024-08-02 23:00:00


Processing 2024-08-02 23:00:00: 100%|██████████| 24/24 [00:18<00:00,  1.29it/s]

Loaded 24 out of 24 slots with dims: FrozenMappingWarningOnValuesAccess({'time': 24, 'lat': 4, 'lon': 6}) in 18.67 seconds 
GeoDataFrame saved for 2024-08-02 as ../downloads/MSG_2ND/kisumu/KISUMU_LST_2024-08-02.gpkg
